<a href="https://colab.research.google.com/github/kevinsconnolly/Big_Data/blob/master/FinalOfficeProductsChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 1s (226 kB/s)
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-09-21 01:28:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.40MB/s    in 0.7s    

2020-09-21 01:28:18 (1.40 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True)

In [ ]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
df.count()

2642434

In [ ]:
# Drop any rows with either null or “not a number” (NaN) values
office_df = df.dropna()

In [ ]:
office_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
office_df.count()

2642195

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz2.cdvvuej8kzy3.us-east-2.rds.amazonaws.com:5432/Challenge"
config = {"user":"postgres2", 
          "password": "passenger8808", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to office_user table in RDS
office_df.write.jdbc(url=jdbc_url, table='office', mode=mode, properties=config)

In [ ]:
Yes_df = office_df.filter(office_df["vine"] == "Y")

In [ ]:
Yes_df.show()

In [ ]:
Yes_df.count()

In [ ]:
No_df = office_df.filter(office_df["vine"] =="N")

In [ ]:
No_df.show()

In [ ]:
No_df.count()

In [ ]:
marketplace_df = office_df.filter(office_df["marketplace"] == "US")

In [ ]:
marketplace_df.count()

In [ ]:
filtered1_df = df.select("customer_id", "product_parent", "product_title", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date").show()

In [ ]:
review_date_df = office_df.filter(office_df["review_date"] == "2015-08-31")

In [ ]:
review_date_df.count()

In [ ]:
#From the date value of 2015-08-31 has 4,157 rows in our 2,642,195 row data frame. This shows our review_date columns holds multiple dates and has no substantical value in our analysis


In [ ]:
filtered2_df = df.select("customer_id", "product_parent", "product_title", "vine", "review_headline", "review_body")

In [ ]:
filtered2_Y_df = filtered2_df.filter(filtered2_df["vine"] == "Y")

In [ ]:
filtered2_Y_df.show()

In [ ]:
#we are doen to 6 columns from our original 15. The vine column is our primary tool for analysis, we have 5 columns left to review. We can tokenize 3 out of the five columns.

In [ ]:
product_title_df = df.select("product_title", "vine")

In [ ]:
product_title_Yes_df = product_title_df.filter(product_title_df["vine"] == "Y")

In [ ]:
product_title_Yes_df.show()

In [ ]:
# Write DataFrame to office_user table in RDS
product_title_Yes.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
review_headline_df = df.select("review_headline", "vine")

In [ ]:
review_headline_Yes_df = review_headline_df.filter(review_headline_df["vine"] == "Y")

In [ ]:
review_headline_Yes_df.show()

In [ ]:
review_body_df = df.select("review_body", "vine")

In [ ]:
review_body_Yes_df = review_body_df.filter(review_body_df["vine"] == "Y")

In [ ]:
review_body_Yes_df.show()

In [ ]:
# Write DataFrame to office_user table in RDS
review_body_Yes_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Import the Tokenizer library.
from pyspark.ml.feature import Tokenizer

In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
	return len(word_list)

In [ ]:

from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:

# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [ ]:
# Create our Tokenize sentences
tokenizer = Tokenizer(inputCol="product_title", outputCol="words")

In [ ]:
# Transform DataFrame
product_title_tokenized = tokenizer.transform(product_title_Yes_df)

In [ ]:
# Select the needed columns and no truncated results
product_title_tokenized.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

In [ ]:
# Create the Tokenized sentences
tokenizer = Tokenizer(inputCol="review_headline", outputCol="words")

In [ ]:
# Transform DataFrame
review_headline_tokenized = tokenizer.transform(review_headline_Yes_df)

In [ ]:
# Select the needed columns and don't truncate results
review_headline_tokenized.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

In [ ]:
# Create our Tokenize sentences
tokenizer = Tokenizer(inputCol="review_body", outputCol="words")

In [ ]:
# Transform DataFrame
review_body_tokenized = tokenizer.transform(review_body_Yes_df)

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz2.cdvvuej8kzy3.us-east-2.rds.amazonaws.com:5432/Challenge."
config = {"user":"postgres2", 
          "password": "passenger8808", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to office_user table in RDS
product_title_Yes_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to office_user table in RDS
office_df.write.jdbc(url=jdbc_url, table='office_review', mode=mode, properties=config)

In [ ]:
# Write DataFrame to office_user table in RDS
office_df.write.jdbc(url=jdbc_url, table='office_review', mode=mode, properties=config)

In [ ]:
# Write DataFrame to office_user table in RDS
office_df.write.jdbc(url=jdbc_url, table='office_review', mode=mode, properties=config)